In [1]:
#default_exp core

In [2]:
#!pip install nbdev
#!pip install gpy

In [3]:
%config Completer.use_jedi = False

# restaurants_timeseries

> Predicts restaurant daily visit counts with Gaussian processes.

In [4]:
#hide
from nbdev.showdoc import *

In [5]:
import os
import pandas as pd
from datetime import datetime
DATAPATH = 'data'

DATA_FILES = {
    "simple_visits": os.path.join(DATAPATH, "air_visit_data.csv"),
    "store_info": os.path.join(DATAPATH, "air_store_info.csv"),
    "reservations": os.path.join(DATAPATH, "air_reserve.csv"),
    "date_info": os.path.join(DATAPATH, "date_info.csv")
}

def read_data():
    data = dict([(name, pd.read_csv(path)) for name, path in DATA_FILES.items()])
    stores = data['store_info'].drop_duplicates() 
    stores['joinkey']= 'k'
    data['date_info']['joinkey'] = 'k'
    data['all_stores_dates'] = (
        stores
        .merge(data['date_info'], on='joinkey')
        .drop('joinkey', axis=1)
        .rename({'calendar_date': 'visit_date'}, axis=1))
    data['visits'] = (
        data['all_stores_dates']
        .merge(data['simple_visits'], on=['air_store_id', 'visit_date'], how='left'))
    for entry in ('simple_visits', 'all_stores_dates', 'store_info', 'date_info'):
        del(data[entry])
    data['visits'].visitors = data['visits'].visitors.fillna(0).astype(int)
    data['visits']['visit_date'] = pd.to_datetime(data['visits']['visit_date'])
    return data

#export
data = read_data()

In [6]:
for name, df in data.items():
    print(name)
    display(df.head())

reservations


,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


visits


,air_store_id,air_genre_name,air_area_name,latitude,longitude,visit_date,day_of_week,holiday_flg,visitors
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2016-01-01,Friday,1,0
1,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2016-01-02,Saturday,1,0
2,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2016-01-03,Sunday,1,0
3,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2016-01-04,Monday,0,0
4,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2016-01-05,Tuesday,0,0


In [7]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_view_data.ipynb.
Converted 02_model_one.ipynb.
Converted index.ipynb.
